# ML-6. Отбор и селекция признаков

## Загрузка данных

In [1]:
# Импортируем библиотеки

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

In [45]:
%%capture
!wget https://www.dropbox.com/s/64ol9q9ssggz6f1/data_ford_price.xlsx

In [2]:
data = pd.read_excel('data/data_ford_price.xlsx') 
data.head()

,price,year,condition,cylinders,odometer,title_status,transmission,drive,size,lat,long,weather
0,43900,2016,4,6,43500,clean,automatic,4wd,full-size,36.471500,-82.483400,59.0
1,15490,2009,2,8,98131,clean,automatic,4wd,full-size,40.468826,-74.281734,52.0
2,2495,2002,2,8,201803,clean,automatic,4wd,full-size,42.477134,-82.949564,45.0
3,1300,2000,1,8,170305,rebuilt,automatic,4wd,full-size,40.764373,-82.349503,49.0
4,13865,2010,3,8,166062,clean,automatic,4wd,NaN,49.210949,-123.114720,NaN


#  Отбор признаков: мотивация

## Предобработка данных

In [5]:
# Выделим числовые признаки, обработаем пропуски
data = data[['price','year', 'cylinders', 'odometer', 'lat', 'long', 'weather']]
data.dropna(inplace = True)
# Разделим выборку на тренировочную и тестовую
y = data['price']
x = data.drop(columns='price')
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=40)

## Обучение модели

In [55]:
# Обучим модель логистической регрессии и выведем предсказание для тестовой выборки
model = LinearRegression()
model.fit(X_train, y_train)
y_predicted = model.predict(X_test)
# Расчитаем метрику MAE для исходной выборки
mae = mean_absolute_error(y_test, y_predicted)
print('MAE для исходной выборки: %.3f' % mae)

MAE для исходной выборки: 4682.957


## Удаление избыточного признака

In [56]:
# Мы выяснили, что у нас присутствует сильная зависимость между lat и weather. Удалим lat, так как этот признак, 
# в отличие от weather, необходимо округлять.
x.drop(columns='lat', axis = 1, inplace = True)

In [57]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=40)

motiv_model = LinearRegression()
motiv_model.fit(X_train, y_train)
y_predicted = motiv_model.predict(X_test)
 
mae = mean_absolute_error(y_test, y_predicted)
print('Мотивация MAE: %.2f' % mae)

Мотивация MAE: 4672.93


#  Отбор признаков: классификация методов

## Метод рекурсивного исключения признаков

Метод рекурсивного исключения признаков (RFE) предполагает выбор признаков путём рекурсивного рассмотрения всё меньших и меньших наборов фичей.

In [58]:
# Импортируем RFE
from sklearn.feature_selection import RFE

In [59]:
# Выделим три наиболее значимых признака:
y = data['price']
x = data.drop(columns='price')
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=40)

estimator = LinearRegression()
rfe_selector = RFE(estimator, n_features_to_select=3, step=1)
rfe_selector = rfe_selector.fit(X_train, y_train)

rfe_cols = rfe_selector.get_feature_names_out()
rfe_cols

array(['year', 'cylinders', 'lat'], dtype=object)

In [60]:
# Разделим выборку из трёх столбцов, выбраных RFE, на тренировочную и тестовую
y = data['price']
x = data[rfe_cols]
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=40)

# Обучим модель логистической регрессии и выведем предсказание для тестовой выборки
mogel = LinearRegression()
mogel.fit(X_train, y_train)
y_predicted = mogel.predict(X_test)
# Расчитаем метрику MAE для RFE выборки
mae = mean_absolute_error(y_test, y_predicted)
print('MAE для RFE выборки: %.3f' % mae)

MAE для RFE выборки: 5096.570


## Метод выбора признаков на основе фильтров

Один из наиболее популярных метода:
- выбор k лучших переменных: SelectKBest

In [3]:
# Импортируем SelectKBest
from sklearn.feature_selection import SelectKBest, f_regression

In [6]:
# Выделим три наиболее значимых признака:
y = data['price']
x = data.drop(columns='price')
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=40)

selector2 = SelectKBest(f_regression, k=3)
selector2.fit(x, y)
 
skb_cols = selector2.get_feature_names_out()
skb_cols

array(['year', 'cylinders', 'odometer'], dtype=object)

In [7]:
# Разделим выборку из трёх столбцов, выбраных SelectKBest, на тренировочную и тестовую
y = data['price']
x = data[skb_cols]
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=40)

# Обучим модель логистической регрессии и выведем предсказание для тестовой выборки
mogel = LinearRegression()
mogel.fit(X_train, y_train)
y_predicted = mogel.predict(X_test)
# Расчитаем метрику MAE для SelectKBest выборки
mae = mean_absolute_error(y_test, y_predicted)
print('MAE для SelectKBest выборки: %.3f' % mae)

MAE для SelectKBest выборки: 4708.946


## Вывод

Результаты расчёта метрик:
- MAE для исходной выборки: 4682.957
- MAE для Мотивации: 4672.93
- MAE для RFE выборки: 5096.570
- MAE для SelectKBest выборки: 4708.946

Как мы видим отбор наиболее значимых признаков увеличивает целевую меткиру, это может происходить из-за того, что модели проще обнаружить важные закономерности на наиболее значимых признаках, не отвлекаясь на менее значимые закономерности в других признаках.

Оба метода отбора значимых признаков показали результат метрики лучше, чем предыдущие методы, самый лучший результат показал метод рекурсивного исключения признаков (RFE).